Primeiro passo importar todas as Libs que vamos utilizar

In [1]:
# Importa as opções do Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium import webdriver
from time import sleep
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'selenium'

Pegando as URLs de todos os jogos

In [51]:
# url da página inicial da academia das apostas
academia = "https://www.academiadasapostasbrasil.com/"

# abrindo o firefox em background
option = Options()
option.headless = False
driver = webdriver.Firefox(options=option)

driver.get(academia)
# sleep (10)
# driver.quit()

for i in range(0,2):
    sleep(5) # colocar uma espera aqui
    driver.find_element(By.XPATH, '//td[contains(@class, "footer")]')
    driver.find_element(By.XPATH, '//td[contains(@class, "footer")]').click()

sleep(3)

len(driver.find_elements(By.XPATH, '//table//tbody//tr[@type="match"]'))


    

187

In [46]:
sleep(3)
html_table = driver.find_element(By.XPATH, '//table[contains(@class, "dskt")]').get_attribute('outerHTML')

def trata_html(input):
    return " ".join(input.split()).replace('> <','><')

html_table = trata_html(html_table)

soup = BeautifulSoup(html_table, 'html.parser')

len(soup.find_all('tr'))

189

oi
